In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_contrast(image, lower=8.0, upper=8.0)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=8.0, upper = 8.0)
    image = tf.image.random_hue(image, max_delta = 8.0)
    image = tf.image.random_saturation(image, lower=8.0, upper=8.0)
    return image, label

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=10,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 14:44:54.389561: {'loss': 1.768230608010292, 'accuracy': 0.45221874, 'val_loss': 2.231731803553879, 'val_accuracy': 0.2695}
Epoch 1 ends at 14:49:09.425743: {'loss': 0.8955073623895645, 'accuracy': 0.6974875, 'val_loss': 0.7892652446297324, 'val_accuracy': 0.7236}
Epoch 2 ends at 14:53:37.429453: {'loss': 0.5853237542152405, 'accuracy': 0.7998125, 'val_loss': 0.6652675561464516, 'val_accuracy': 0.7738}
Epoch 3 ends at 14:58:04.962555: {'loss': 0.4018213194310665, 'accuracy': 0.86134374, 'val_loss': 0.61259407754157, 'val_accuracy': 0.803}
Epoch 4 ends at 15:02:30.885906: {'loss': 0.28038163009658457, 'accuracy': 0.9030875, 'val_loss': 0.6519152954885155, 'val_accuracy': 0.7978}
Epoch 5 ends at 15:07:05.573655: {'loss': 0.2111654236845672, 'accuracy': 0.92725, 'val_loss': 0.7919054792565146, 'val_accuracy': 0.7838}
Epoch 6 ends at 15:11:36.643460: {'loss': 0.16636683169305325, 'accuracy': 0.94290626, 'val_loss': 0.7293842630401538, 'val_accuracy': 0.8078}
Epoch 7 ends at

In [12]:
model.evaluate(test_data)

157/157 [==============================].4243 - accuracy: 0.89 - 0s 68ms/step - loss: 0.5417 - accuracy: 0.85 - 0s 63ms/step - loss: 0.7401 - accuracy: 0.82 - 0s 62ms/step - loss: 0.6990 - accuracy: 0.83 - 0s 62ms/step - loss: 0.6547 - accuracy: 0.84 - 0s 59ms/step - loss: 0.5967 - accuracy: 0.84 - 0s 57ms/step - loss: 0.5902 - accuracy: 0.85 - 0s 54ms/step - loss: 0.6311 - accuracy: 0.84 - 0s 53ms/step - loss: 0.6254 - accuracy: 0.84 - 1s 52ms/step - loss: 0.6340 - accuracy: 0.83 - 1s 51ms/step - loss: 0.6180 - accuracy: 0.84 - 1s 50ms/step - loss: 0.6274 - accuracy: 0.83 - 1s 49ms/step - loss: 0.6317 - accuracy: 0.83 - 1s 48ms/step - loss: 0.6371 - accuracy: 0.83 - 1s 48ms/step - loss: 0.6468 - accuracy: 0.83 - 1s 47ms/step - loss: 0.6864 - accuracy: 0.83 - 1s 47ms/step - loss: 0.6834 - accuracy: 0.83 - 1s 46ms/step - loss: 0.7022 - accuracy: 0.82 - 1s 46ms/step - loss: 0.6979 - accuracy: 0.82 - 1s 46ms/step - loss: 0.6962 - accuracy: 0.82 - 1s 46ms/step - loss: 0.6764 - accuracy: 0.

[0.7598808333752262, 0.8096]

In [13]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))